# Flood Estimation - Newcastle University

This notebook was created to capture the flood estimation process for single site and pool as described by the FEH manual

---

#### Author: 
                Luis F Patino Velasquez - NU
                l.f.patino-velasquez2@ncl.ac.uk

#### Date:
                Feb 2022

#### Version:
                1.0

#### Notes:
                - To get jupyter env version type `!jupyter --version` in a python cell
            
#### Jupyter version:
                jupyter core     : 4.6.3
                jupyter-notebook : 6.4.8
                qtconsole        : 5.2.2
                ipython          : 7.18.1
                ipykernel        : 5.3.4
                jupyter client   : 6.1.7
                nbconvert        : 6.4.1
                ipywidgets       : 7.6.5
                nbformat         : 5.1.3
                traitlets        : 5.0.5

#### Python version:
                3.8.5 

---

## Main considerations

* This script uses the [NRFA API](https://nrfaapps.ceh.ac.uk/nrfa/nrfa-api.html#ws-station-info) to retrieved the data necessary for the analysis




# Notebook set-up

## 1. Setting Python Modules



In [ ]:
# %pip install git+https://github.com/OpenHydrology/lmoments3.git
# Data handling
import pandas as pd
import numpy as np

# Distribution libraries
%pip install git+https://github.com/OpenHydrology/lmoments3.git
import lmoments3 as lm
from lmoments3 import distr
from scipy.stats import genextreme, genlogistic


# Data fetching
import urllib3
from urllib3 import request
import json
import certifi # to handle certificate verification - just to make sure it is a safe site
from google.colab import files

# # Notebook interaction
from IPython.display import display, clear_output
from ipywidgets import widgets, Dropdown, Layout

# Data visualisations
# from matplotlib import pyplot as plt
import bokeh.io
bokeh.io.output_notebook()
import bokeh.plotting
import numpy as np
from bokeh.models import LinearAxis, Range1d
from bokeh.models import FuncTickFormatter
%pip install ipyleaflet
%pip install geopandas
from ipyleaflet import *
import geopandas


## 2. Global variables

In [ ]:
# The base URL to access the NFRA API
base_url = "https://nrfaapps.ceh.ac.uk/nrfa/ws"
base_url_timeseries = "https://nrfaapps.ceh.ac.uk/nrfa/ws/time-series"
# for all available formats - as NFRA API documentation
response = "format=json-object"

## 3.Functions

In [ ]:
############################################################
# Create dataframe in the right format
############################################################
def create_df(df_api):
    """This takes the JSON normalised dataframe and create 
    a dataframe with two column, one for the date and one for
    values

    Args:
        df_api (object): JSON normalized panda dataframe

    Returns:
        object: dataframe containing the NRFA API data
    """
    # Adding column name
    df_api.columns = ['data']
    
    # The data comes all together so needs splitting
    # even rows - dates / odd rows - data values
    df_even = df_api.iloc[:-2:2]
    lst_even = list(df_even['data']) # column to list 
    df_odd = df_api.iloc[1:-2:2]
    lst_odd = list(df_odd['data']) # column to list 
    
    # Create dataframe with the available data
    zipped = list(zip(lst_even, lst_odd))
    df_fixed = pd.DataFrame(zipped, columns=['Date', 'Flow'])
    
    return df_fixed
    
def fetch_station_data(url_to_fetch):
    """Fetch data using NRFA API

    Args:
        url_to_fetch (string): url part containing the information
        to be requested

    Returns:
        object: dataframe containing the NRFA API data
    """
    # Get data from the API
    station_data_url = f"{base_url_timeseries}/" + f"?{response}" + f"&{url_to_fetch}"
    r = http.request('GET', station_data_url)
    r.status

    # Decode json data into a dict object
    data = json.loads(r.data.decode('utf-8'))
    
    # In this dataset, the data to extract is under 'station-ids'
    df = pd.DataFrame() # always start with an empty dataframe
    df = pd.json_normalize(data, 'data-stream') # this can be a parameter to parse
    # Check if station has any data for the variable of interest
    # If not data then create dataframe fill with zeros
    if df.empty is False:
        df_final = create_df(df)
    else:
        df_final = pd.DataFrame(0, index=np.arange(1), columns=['Date','Flow'])
    
    # Add a name to the dataframe
    name_df = url_to_fetch.split('=')[1].split('&')[0]
    df_final.name = name_df
    
    return df_final

def station_id_wdg(url_to_fetch):
    """Build widget to select NRFA station id

    Args:
        url_to_fetch (string): the url parameter as per NRFA API docs
    """
    # Get data from the API
    station_ids_url = f"{base_url}/" + url_to_fetch + f"?{response}"
    r = http.request('GET', station_ids_url)
    r.status

    # Decode json data into a dict object
    data = json.loads(r.data.decode('utf-8'))

    # In this dataset, the data to extract is under 'station-ids'
    df_stationIDs = pd.json_normalize(data, 'station-ids')
    df_stationIDs.columns = ['station-ids']

    # Build the widget to be displayed
    
    stations_dropdown = widgets.Dropdown(
        options=list(df_stationIDs['station-ids']),
        value=22007,
        width='auto', 
        height='auto',
        description='Station ID:',
        disabled=False,
    )
    return stations_dropdown

def nrfa_data(b):
    """Using the station id from the dropdown
    menu. It uses the NRFA API to get the data
    in a JSON format and put it inside a pandas
    dataframe

    Args:
        id_station (int): station id from dropdown

    Returns:
        [object]: dataframe containing the API data
    """
    
    # read station id from dropdown
    nrfa_data.stn_id = station_wdg.value
    
    # add the missing part of the API url
    station_API_url_amx = f'data-type=amax-flow&station={nrfa_data.stn_id}'
    station_API_url_pot = f'data-type=pot-flow&station={nrfa_data.stn_id}'
    
    # fetch data and create dataframes
    nrfa_data.data_api_amx = fetch_station_data(station_API_url_amx)
    nrfa_data.data_api_pot = fetch_station_data(station_API_url_pot)
    
    # with output:
    #     print (f'Your data for station {stn_id} has been fetched from the NRFA API')
    #     print(nrfa_data.data_api_amx)
    
    lst_df_fetched = [nrfa_data.data_api_amx, nrfa_data.data_api_pot]

    with output:
        clear_output()
        for df_fetched in lst_df_fetched:         
            if df_fetched.shape[0] == 1:
                print('----'*4)
                print (f'No data could be found in the {df_fetched.name} for the station {nrfa_data.stn_id} in the NRFA API')
            else:
                print('----'*4)
                print (f'{df_fetched.name} data for the station {nrfa_data.stn_id} has been put into a datrame ready for analysis')
    # print (f'Your data for station {stn_id} has been fetched from the NRFA API')
    return nrfa_data.data_api_amx, nrfa_data.data_api_pot

def qmed_single_site_amax(amx_dataframe):
    """QMED calculation using annual maximum flow records (amax)

    Args:
        amx_dataframe (object): dataframe with amax records from NRFA

    Raises:
        InsufficientDataError: records in dataframe are less than 2

    Returns:
        float : QMED value
    """
    n = amx_dataframe.shape
    if n[0] < 2:
        raise InsufficientDataError(f"Insufficient annual maximum flow records available for catchment")
    
    return amx_dataframe['Flow'].median()

def get_lmoment(amax_data):
    """Get lmoment using annual maxima flow (amax) data for single sites

    Args:
        amax_data (object): pandas dataframe with amax data

    Returns:
        float: L-CV and L-SKEW
    """
    # Calculate L-CV and L-SKEW for a gauged catchment. Uses `lmoments3` library.
    l1, l2, t3 = lm.lmom_ratios(amax_data.to_numpy(), nmom=3)
    l_cv = l2 / l1
    l_skew = t3
    return [l_cv, l_skew]

def fetch_descriptors(url_to_fetch):
    """Fetch data using NRFA API

    Args:
        url_to_fetch (string): url part containing the information
        to be requested

    Returns:
        object: dataframe containing the NRFA API data 
    """
    # Get data from the API
    data_url = f"{base_url}/" + f"{url_to_fetch}"
    r = http.request('GET', data_url)
    r.status

    # Decode json data into a dict object
    data = json.loads(r.data.decode('utf-8'))
    
    # In this dataset, the data to extract is under 'station-ids'
    df = pd.DataFrame() # always start with an empty dataframe
    df = pd.json_normalize(data, 'data') # this can be a parameter to parse
    
    # Check if station has any data for the variable of interest
    # If not data then create dataframe fill with zeros
    if df.empty is False:
      return df
    else:
      return f'There was an error fetching the data, please check that this url is working: {data_url}'
  
def nrfa_data_pool(id_station):
  """Using the station id from the dropdown
  menu. It uses the NRFA API to get the data
  in a JSON format and put it inside a pandas
  dataframe

  Args:
      id_station (int): station id from dropdown

  Returns:
      [object]: dataframe containing the API data
  """
  
  # read station id from dropdown
  nrfa_data_pool.stn_id = id_station
  
  # add the missing part of the API url
  station_API_url_amx = f'data-type=amax-flow&station={nrfa_data_pool.stn_id}'
  
  # fetch data and create dataframes
  nrfa_data_pool.data_api_amx = fetch_station_data(station_API_url_amx)

  # Get moments using the data fetch from NRFA
  l_cv = get_lmoment(nrfa_data_pool.data_api_amx['Flow'])[0]
  l_skew = get_lmoment(nrfa_data_pool.data_api_amx['Flow'])[1]

  # Get number of records
  n = nrfa_data_pool.data_api_amx.shape[0]
  
  return [l_cv, l_skew, n]

def graph_single_site(dataframe_amax, distrib_name, dist_param):
    """Plotting Annual maxima flow (amax) data and fitted distribution

    Args:
        dataframe_amax (object): dataframe with amax data
        distrib_name (str): distribution name, only 'gev' and 'glo' can be used
        dist_param (OrderedDict): fitted curve parameteres
    """
    # Create dataframe for the observed data
    # Calculating the reduced variate requires the rank of the measurement (i.e. is it the 4th smallest value?). 
    # We use the rank function to calculate this. ascending=False as the smallest value should have rank=1. 
    # method=first means that there are no tied ranks
    dataframe_amax["obs_rank"] = dataframe_amax['Flow'].rank(method="first", ascending=True)
    # This gives us the total number of observations 
    num_obs = dataframe_amax["obs_rank"].max()
    # The non-exceedence probability, a step to calculate the reduced variate, y.
    dataframe_amax["F"] = (dataframe_amax["obs_rank"] - 0.44)/(dataframe_amax["obs_rank"].max() + 0.12)
    # calculating the reduced variate
    dataframe_amax["y"] = -1*np.log(-1*np.log(dataframe_amax .F)) 

    if distrib_name.lower() == 'gev':
      # Create dataframe with fitted GEV
      sim_df = pd.DataFrame({"F": np.linspace(0.01, 0.99, 99)}) #create a new dataframe for the simulated values
      sim_df["y"] = -1*np.log(-1*np.log(sim_df.F))# create the reduced variates we want to estimate flows for
      sim_df["GEV"] = 1*(dist_param['loc']+(dist_param['scale']*\
                      (1-(-1*np.log(sim_df.F))**dist_param['c'])/dist_param['c'])) #estimate the flows using the fitted GEV parameters that we calculated earlier
      sim_df["GEV"] = sim_df["GEV"] * qmed_single_site

      # # Create figure
      # Add data to figure
      f = bokeh.plotting.figure(title="simple line example")
      f.circle(dataframe_amax.y, dataframe_amax.Flow, size=8, color='#35B778', legend_label='Obs')
      f.line(sim_df.y, sim_df.GEV, line_width=3, alpha=0.6, color='#41b6c4', legend_label='GEV')

      # Second axis
      f.extra_y_ranges = {'y2': Range1d(start=-2, end=6)}
      f.add_layout(LinearAxis(y_range_name='y2', axis_label="Return period"), "above")

      # Setting format of figure
      # f.x_range = Range1d(-2, 6)
      f.legend.location = "top_left"
      f.xaxis.axis_label = "Reduced variate"
      f.yaxis.axis_label = r"Flow (m^3/s)"
      #change ticks in the second axis
      f.xaxis[0].ticker = [0.37, 1.5, 2.25, 3.9, 4.6, 5.3]
      #change labels for the ticks in the second axis
      f.xaxis[0].formatter = CustomFuncTickFormatter = FuncTickFormatter(code = '''
      return {0.37:2, 1.5:5, 2.25:10, 3.9:50, 4.6:100, 5.3:200}
      [tick]
      ''')

      return bokeh.plotting.show(f)

def set_pool(b):
    """Set dataframe with data to be used in the pool process

    Args:
        b ([type]): widget button listener

    Returns:
        object: pandas dataframe
    """
    # read station id from dropdown
    set_pool.stn_id = pool_wdg.value

    # Use ID to set the dataframe
    pool_options = station_pool_combin.loc[((station_pool_combin['id'] == set_pool.stn_id) & (station_pool_combin['dist_pool'] <= 0.5))]

    # Sort dataframe to make sure the station are look at in order
    set_pool.pool_options_sorted = pool_options.sort_values(by='dist_pool')

    print(' The dataframe has been set for the pooling')
    return set_pool.pool_options_sorted

def graph_pool(dataframe_amax, distrib_name, lst_dist_param):
    """Create graph displaying Annual maxima data and fitted curves for
    single site and pool analysis

    Args:
        dataframe_amax (object): pandas dataframe with amax data for the station of interest
        distrib_name (str): distribution name, only 'gev' and 'glo' supported
        lst_dist_param (list): list containing the OrderDict for the fitter parameter curves from single site and pool analasis
    """
    # Create dataframe for the observed data
    # Calculating the reduced variate requires the rank of the measurement (i.e. is it the 4th smallest value?). 
    # We use the rank function to calculate this. ascending=False as the smallest value should have rank=1. 
    # method=first means that there are no tied ranks
    dataframe_amax["obs_rank"] = dataframe_amax['Flow'].rank(method="first", ascending=True)
    # This gives us the total number of observations 
    num_obs = dataframe_amax["obs_rank"].max()
    # The non-exceedence probability, a step to calculate the reduced variate, y.
    dataframe_amax["F"] = (dataframe_amax["obs_rank"] - 0.44)/(dataframe_amax["obs_rank"].max() + 0.12)
    # calculating the reduced variate
    dataframe_amax["y"] = -1*np.log(-1*np.log(dataframe_amax .F)) 

    if distrib_name.lower() == 'gev':
        # Create dataframe with fitted GEV for single site
        #create a new dataframe for the simulated values
        sim_df_single = pd.DataFrame({"F": np.linspace(0.01, 0.99, 99)}) 
        # create the reduced variates we want to estimate flows for
        sim_df_single["y"] = -1*np.log(-1*np.log(sim_df_single.F))
        #estimate the flows using the fitted GEV parameters that we calculated earlier
        sim_df_single["GEV"] = 1*(lst_dist_param[0]['loc']+(lst_dist_param[0]['scale']*\
                        (1-(-1*np.log(sim_df_single.F))**lst_dist_param[0]['c'])/lst_dist_param[0]['c']))
        sim_df_single["GEV"] = sim_df_single["GEV"] * qmed_single_site

        # Create dataframe with fitted GEV for pool
        sim_df_pool = pd.DataFrame({"F": np.linspace(0.01, 0.99, 99)})
        sim_df_pool["y"] = -1*np.log(-1*np.log(sim_df_pool.F))
        sim_df_pool["GEV"] = 1*(lst_dist_param[1]['loc']+(lst_dist_param[1]['scale']*\
                        (1-(-1*np.log(sim_df_pool.F))**lst_dist_param[1]['c'])/lst_dist_param[1]['c']))
        sim_df_pool["GEV"] = sim_df_pool["GEV"] * qmed_single_site

        # # Create figure
        # Add data to figure
        f = bokeh.plotting.figure(title="Single Site and Pool Flood Estimation")
        f.circle(data_amax.y, data_amax.Flow, size=8, color='#35B778', legend_label='Obs')
        f.line(sim_df_single.y, sim_df_single.GEV, line_width=3, alpha=0.6, color='#225ea8', legend_label='GEV - Single site')
        f.line(sim_df_pool.y, sim_df_pool.GEV, line_width=3, alpha=0.6, color='#41b6c4', legend_label='GEV - Pool')

        # Second axis
        f.extra_y_ranges = {'y2': Range1d(start=-2, end=6)}
        f.add_layout(LinearAxis(y_range_name='y2', axis_label="Return period"), "above")

        # Setting format of figure
        # f.x_range = Range1d(-2, 6)
        f.legend.location = "top_left"
        f.xaxis.axis_label = "Reduced variate"
        f.yaxis.axis_label = r"Flow (m^3/s)"
        #change ticks in the second axis
        f.xaxis[0].ticker = [0.37, 1.5, 2.25, 3.9, 4.6, 5.3]
        #change labels for the ticks in the second axis
        f.xaxis[0].formatter = CustomFuncTickFormatter = FuncTickFormatter(code = '''
        return {0.37:2, 1.5:5, 2.25:10, 3.9:50, 4.6:100, 5.3:200}
        [tick]
        ''')

        return bokeh.plotting.show(f)


### Handling Certification - require for Google Colab


In [ ]:
# handle certificate verification and SSL warnings
# https://urllib3.readthedocs.io/en/latest/user-guide.html#ssl
http = urllib3.PoolManager(
       cert_reqs='CERT_REQUIRED',
       ca_certs=certifi.where())

#Get Annual maxima flow (amax) data
This process can be done by using either one of the two methods below:


##Using NRFA API
This will enable the user to fetch the data from the NRFA API

In [ ]:
# Create widget for station and button to click for submission

# List of stations id widget
station_API_url = 'station-ids'
station_wdg = station_id_wdg(station_API_url)
    


# Create button widget ready to feath data
button = widgets.Button(description="Click to get data from NRFA", 
                        layout=widgets.Layout(height='50px', width='auto'), button_style="info")
output = widgets.Output() 
button.on_click(nrfa_data)

# widgets.Box(children=[station_wdg,button, output])
widgets.TwoByTwoLayout(top_right=button,
                       top_left=station_wdg,
                       bottom_left=output)

TwoByTwoLayout(children=(Dropdown(description='Station ID:', index=234, layout=Layout(grid_area='top-left'), o…

---
---



## **Use the code below if you are uploading a file**

##Uploading csv file
Run the cell below to enable the file upload option

In [ ]:
amax_data_file = files.upload()

## Parse csv file into a panda dataframe ready for analysis
<mark> Only run the cell below if you have uploaded a file<mark>

In [ ]:
data_api_amx = pd.read_csv('/content/' + list(amax_data_file.keys())[0])
print('-------------------------------------')
print(f'The csv data has been uploaded and it is set for analysis \nyou can use print(nrfa_data.data_api_amx) to see the dataframe')
print('-------------------------------------')

---
---
# Single site flood estimation
In single-site analysis only the data from the subject site are used. The recommended procedure is to treat the problem in two steps:



## 1. QMED - Using Annual Maximum Values

In [ ]:
data_source =  input('Are you using NRFA or CSV: ')

if data_source.lower() == 'nrfa':
  data_amax = nrfa_data.data_api_amx.copy()
  qmed_single_site = qmed_single_site_amax(data_amax)
  station_id = nrfa_data.stn_id
else:
  data_amax = data_api_amx
  qmed_single_site = qmed_single_site_amax(data_amax)
  station_id = 'Predictions'

print('-------------------------------------')
print(f'The QMED for Station: {station_id} is: {qmed_single_site}')
print('-------------------------------------')

Are you using NRFA or CSV: NRFA
-------------------------------------
The QMED for Station: 23001 is: 841.8109999999999
-------------------------------------


## 2. Derivation of the growth curve
In most situations, use of a Generalised Logistic growth curve is recommended for UK flood data.

### 2.1. Calculate L-CV and L-SKEW for a gauged catchment. (L-moment)
- Estimation of the growth curve parameters is achieved using an L-moment method in the sample dataset

This part is based on the process defined by the **Single Site** method `class GrowthCurveAnalysis(Analysis)` in the Open Hydrology code



In [ ]:
station_lmoment = get_lmoment(data_amax['Flow'])
l_cv = station_lmoment[0]
l_skew = station_lmoment[1]

print('-------------------------------------')
print(f'this is the L-CV: {l_cv}')
print(f'this is the L-SKEW: {l_skew}')
print('-------------------------------------')


-------------------------------------
this is the L-CV: 0.16111543041168574
this is the L-SKEW: 0.14746479571125987
-------------------------------------


### 2.2. Fitting distribution using L-moment ratios 
This workflow is only for GEV
- Section 15.2.2 FEH manual vol 3

In [ ]:
param = distr.gev.lmom_fit(lmom_ratios=[1, l_cv, l_skew])
param

OrderedDict([('c', 0.035259444525624656),
             ('loc', 0.8696374605059898),
             ('scale', 0.23988893334171071)])

### 2.3. Flood estimation
- This user can input one or several return periods. 2 or more return periods needs to be entered separated by commas
- Return periods will be changed to the annual exceedance probability (AEP) by the code. 

| Exceedance Probability  (AEP) | Potential Frequency |
| ----------------------------- |---------------------|
| 1% AEP                        | 1 in 100 Years      |
| 2% AEP                        | 1 in 50 Years       |
| 5% AEP                        | 1 in 20 Years       |
| 10% AEP                       | 1 in 10 Years       |
| 20% AEP                       | 1 in 5 Years        |
| 50% AEP                       | 1 in 2 Years        |
| 100% AEP                      | Happens Every Year  |

- If one return period is given then a single value will be return, otherwise list of values is returned. The first value of the list will be the output of the first return period and so on for other return values.

In [ ]:
# Asking for return period and creating the aep list
# The error checking might need to be improved!!! 
T = []
aep = []
try:
  # ask use for input
  T = [int(item) for item in input("If entering more than one return period please use commas i.e 100,50,20 : ").split(',')]
  # change return period to aep
  aep = [1/item for item in T]
  print('-------------------------------------')
  print(f'The list of aep you have entered is: {aep}')
  print('-------------------------------------')
except:
  print('-------------------------------------')
  print('You have not entered a valid value. Please, re-run this cell and make sure of using commas if you are entering more than one return period')
  print('-------------------------------------')


If entering more than one return period please use commas i.e 100,50,20 : 500,100,50
-------------------------------------
The list of aep you have entered is: [0.002, 0.01, 0.02]
-------------------------------------


### Flood estimation

In [ ]:
# Taken the parameters of the fit distribution
est_flow = genextreme.ppf(1 - np.array(aep), **param)
# Multiply the est_flow by QMED to obtained the final flood estimation
final_flows = est_flow * qmed_single_site
print('-------------------------------------')
print(f'The Flood Estimation for Station: {station_id} = {final_flows}')
print('-------------------------------------')

-------------------------------------
The Flood Estimation for Station: 23001 = [1858.91080158 1589.60505676 1468.23095323]
-------------------------------------


### 2.4. Graphical Representation

In [ ]:
graph_single_site(data_amax, 'gev', param)

# Flood Estimation - Pool Method

<mark> This analysis can only be done with data taken from NRFA. You will not be able to run the code below using the simulation data<mark>

The fundamental idea in obtaining the pooling-group is to select a group of
sites that are hydrologically similar to the subject site.
There are two main issues involved in form ing pooling-groups: finding
similar sites and choosing how many sites to include


## 1. Getting the descriptor for all the catchments

The catchment pool here is going to be based on a region of influence approach

In [ ]:
# Create dataframe containing all the stations with the descriptors

# add the missing part of the API url for descriptors
station_nrfa_descrip = f'station-info?station=*&format=json-object&fields=all'
# fetch data and create dataframes
station_nrfa_descrip = fetch_descriptors(station_nrfa_descrip)

# create dataframe to check for cathcment pool
station_pool_all = station_nrfa_descrip[['id', 'latitude', 'longitude', 'catchment-area','feh-pooling', 'feh-qmed', 'saar-1961-1990', 'bfihost']]

# select rows (stations) where the feh-pooling is set to true
# this is needed to avoid errors when calculation ln for the distance
station_pool = station_pool_all[station_pool_all['feh-pooling'] == True]

#Create a dataframe with all possible combination between stations
# Using merge to generate all possibilities between all the stations 
station_pool_combin = pd.merge(station_pool.assign(key=0), station_pool.assign(key=0),suffixes=('', '_stat_2') , on='key').drop('key', axis=1)

# Calculate the distance between stations as per FEH formula 16.3 pg 158
station_pool_combin['AREA_term'] = 0.5*((np.log(station_pool_combin['catchment-area']) - np.log(station_pool_combin['catchment-area_stat_2']))/1.34)**2
station_pool_combin['SAAR_term'] = ((np.log(station_pool_combin['saar-1961-1990']) - np.log(station_pool_combin['saar-1961-1990_stat_2']))/0.38)**2
station_pool_combin['BFIHOST_term'] = ((station_pool_combin['bfihost'] - station_pool_combin['bfihost_stat_2'])/0.15)**2
station_pool_combin['dist_pool'] = np.sqrt(station_pool_combin['AREA_term'] + station_pool_combin['SAAR_term'] + station_pool_combin['BFIHOST_term'])

station_pool_combin

,id,latitude,longitude,catchment-area,feh-pooling,feh-qmed,saar-1961-1990,bfihost,id_stat_2,latitude_stat_2,longitude_stat_2,catchment-area_stat_2,feh-pooling_stat_2,feh-qmed_stat_2,saar-1961-1990_stat_2,bfihost_stat_2,AREA_term,SAAR_term,BFIHOST_term,dist_pool
0,2001,58.1410,-3.70295,551.4,True,True,1117.0,0.324,2001,58.14100,-3.70295,551.4,True,True,1117.0,0.324,0.000000,0.000000,0.000000,0.000000
1,2001,58.1410,-3.70295,551.4,True,True,1117.0,0.324,2002,58.01056,-3.87757,434.4,True,True,1217.0,0.351,0.015839,0.050912,0.032400,0.314882
2,2001,58.1410,-3.70295,551.4,True,True,1117.0,0.324,3002,57.89321,-4.54668,241.1,True,True,1784.0,0.436,0.190560,1.518158,0.557511,1.505400
3,2001,58.1410,-3.70295,551.4,True,True,1117.0,0.324,3003,57.96185,-4.70086,330.7,True,True,1896.0,0.359,0.072782,1.938689,0.054444,1.437329
4,2001,58.1410,-3.70295,551.4,True,True,1117.0,0.324,4006,57.59587,-5.00577,116.1,True,True,2203.0,0.333,0.675927,3.194437,0.003600,1.968239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306911,236007,54.3084,-7.68602,166.3,True,True,1330.0,0.495,205020,54.55465,-5.74545,61.8,True,True,933.0,0.438,0.272856,0.870435,0.144400,1.134765
306912,236007,54.3084,-7.68602,166.3,True,True,1330.0,0.495,206001,54.21668,-6.33695,120.3,True,True,975.0,0.569,0.029196,0.667647,0.243378,0.969650
306913,236007,54.3084,-7.68602,166.3,True,True,1330.0,0.495,206006,54.13925,-5.93509,13.8,True,True,1704.0,0.336,1.725257,0.425240,1.123600,1.809446
306914,236007,54.3084,-7.68602,166.3,True,True,1330.0,0.495,236005,54.27093,-7.49124,313.6,True,True,1156.0,0.421,0.112043,0.136148,0.243378,0.701119


## 2. Add the distance based on the FEH Manual - Formula 16.3 pg 158
Get pool options based on the station id
distance measure ranges from 0 to 6, but is typically about 0.5 for stations within a pooling group - FEH pg 158

In [ ]:
# Create widget for station and button to click for submission

# List of stations id widget
station_API_url = 'station-ids'
pool_wdg = station_id_wdg(station_API_url)
    


# Create button widget ready to feath data
button = widgets.Button(description="Click to set the pool", layout=widgets.Layout(height='50px', width='auto'), button_style="info")
output = widgets.Output() 
button.on_click(set_pool)

widgets.TwoByTwoLayout(top_right=button,
                       top_left=pool_wdg,
                       bottom_left=output)

TwoByTwoLayout(children=(Dropdown(description='Station ID:', index=234, layout=Layout(grid_area='top-left'), o…

 The dataframe has been set for the pooling


## 3. Stations to be used as part of the Pool
Run the cell below to see the dataframe containing the data that will be used for the pool analysis. Field ```id_stat_2``` display the catchments id's for the pool

In [ ]:
set_pool.pool_options_sorted

,id,latitude,longitude,catchment-area,feh-pooling,feh-qmed,saar-1961-1990,bfihost,id_stat_2,latitude_stat_2,longitude_stat_2,catchment-area_stat_2,feh-pooling_stat_2,feh-qmed_stat_2,saar-1961-1990_stat_2,bfihost_stat_2,AREA_term,SAAR_term,BFIHOST_term,dist_pool
49395,23001,54.94994,-1.9413,2175.6,True,True,1016.0,0.318,23001,54.94994,-1.94130,2175.6,True,True,1016.0,0.318,0.000000,0.000000,0.000000,0.000000
49397,23001,54.94994,-1.9413,2175.6,True,True,1016.0,0.318,23003,55.05348,-2.14938,1007.5,True,True,1023.0,0.310,0.165026,0.000326,0.002844,0.410118
49420,23001,54.94994,-1.9413,2175.6,True,True,1016.0,0.318,25009,54.48898,-1.43897,1264.0,True,True,966.0,0.374,0.082110,0.017636,0.139378,0.489003


## 4.Geographical location of the pool stations



In [ ]:
# Create Map
m = Map(center=(54.5,1), zoom=5, basemap=basemaps.OpenStreetMap.Mapnik)

# Change dataframe to geopandas so that it can be added to the map
gdf = geopandas.GeoDataFrame(set_pool.pool_options_sorted, geometry=geopandas.points_from_xy
                             (set_pool.pool_options_sorted.longitude_stat_2, set_pool.pool_options_sorted.latitude_stat_2))

# Add pool stations to the map
geo_data = GeoData(geo_dataframe = gdf,
    style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name = 'Release')

m.add_layer(geo_data)
m


Map(center=[54.5, 1], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

## 5. Pool - Distance table
This process calculates the distance value needed to determine the order of the pool. All calculation are based on Formula 7.1 and 7.2 FEH Manual pg 40



In [ ]:

# Get pool options based on the station id
pool_options_sorted = set_pool.pool_options_sorted

# create list with catchment in the pool to look at the records until it matches the 5T rules
station_pool_lst = pool_options_sorted['id_stat_2'].tolist()

# get the data for each of the stations in the pool
# pool_df = pd.DataFrame(columns=['Date','Flow']) # this will store the data to be use in the pool
# station_pool_lst = [22007, 23007, 27092]
lst_lcv_pool = []
lst_skw_pool = []
lst_cnt_pool = []
for station in station_pool_lst:
  temp_station_amax = nrfa_data_pool(station)
  lst_lcv_pool.append(temp_station_amax[0])
  lst_skw_pool.append(temp_station_amax[1])
  lst_cnt_pool.append(temp_station_amax[2])

pool_df_info = pd.DataFrame({'id':station_pool_lst, 'n_records': lst_cnt_pool,
                             'lcv':lst_lcv_pool,'lskew':lst_skw_pool})


# Calculate weights - based on example 7.1 of FEH - page 41

# 1. Add similarity ranking column
total_n = pool_df_info['n_records'].sum()
for i in range(pool_df_info.shape[0]):
  # the last value of the table needs to be the same as n_records - FEH example 7.1
  if i == pool_df_info.shape[0] - 1:
    pool_df_info.loc[i,'Sim_rank'] = pool_df_info.loc[i,'n_records'] / pool_df_info['n_records'].sum()
  else:
    pool_df_info.loc[i,'Sim_rank'] = total_n / pool_df_info['n_records'].sum()
    total_n = total_n - pool_df_info.loc[i+1,'n_records']
  
# 2. Add Weight
pool_df_info['weight'] = pool_df_info['Sim_rank']  * pool_df_info['n_records'] 
pool_df_info['relat_weight'] = pool_df_info['weight'] / pool_df_info['weight'].sum()

### 5.1. Run the cell below to look at the pool distance table

In [ ]:
pool_df_info

,id,n_records,lcv,lskew,Sim_rank,weight,relat_weight
0,23001,64,0.161115,0.147465,1.000000,64.000000,0.546684
1,23003,60,0.178319,0.207121,0.653179,39.190751,0.334765
2,25009,49,0.178077,-0.034361,0.283237,13.878613,0.118550


### 5.2. Pool selection
This process uses the 5*T rule to select the stations that will take part in the pool analysis

In [ ]:
T =  int(input('What is your return period? - this is the number of years you are interested in: '))
records_needed = 5*T + 10 # +10 it is to ensure that anything that is at least 10 values above gets added to the pool
# Create copy of the pool table to avoid any errors
pool_final = pool_df_info.copy()
# Get the cummulative sum of records for each station
pool_final ['cumsum'] = pool_final ['n_records'].cumsum()
pool_final  = pool_final [pool_final ['cumsum'] <= records_needed]

print('-------------------------------------')
print('\nThis is the final pool size:\n')
print('-------------------------------------')

pool_final

What is your return period? - this is the number of years you are interested in: 100
-------------------------------------

This is the final pool size:

-------------------------------------


,id,n_records,lcv,lskew,Sim_rank,weight,relat_weight,cumsum
0,23001,64,0.161115,0.147465,1.000000,64.000000,0.546684,64
1,23003,60,0.178319,0.207121,0.653179,39.190751,0.334765,124
2,25009,49,0.178077,-0.034361,0.283237,13.878613,0.118550,173


## 5.3.Geographical location of the final pool

In [ ]:
# Get coordinates ready for the map using the final pool table
pool_map = pool_final.merge(set_pool.pool_options_sorted,left_on='id', right_on='id_stat_2', how='left')
pool_map = pool_map[['id_stat_2', 'latitude_stat_2',	'longitude_stat_2']]

# Create Map
mp = Map(center=(54.5,1), zoom=5, basemap=basemaps.OpenStreetMap.Mapnik)

# Change dataframe to geopandas so that it can be added to the map
gdf = geopandas.GeoDataFrame(pool_map, geometry=geopandas.points_from_xy
                             (pool_map.longitude_stat_2, pool_map.latitude_stat_2))

# Add pool stations to the map
geo_data_mp = GeoData(geo_dataframe = gdf,
    style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name = 'Release')

mp.add_layer(geo_data_mp)
mp

Map(center=[54.5, 1], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

### 5.1. Calculate L-CV pooled and L-SKEW pooled

In [ ]:
# 1. Add L-CV * Weight and L-SKEW * Weight columns - Formula 7.1 and 7.2 FEH Manual pg 40
pool_final['lcv_x_weight'] = pool_final['weight'] * pool_final['lcv']
pool_final['lskew_x_weight'] = pool_final['weight'] * pool_final['lskew']

pool_final

,id,n_records,lcv,lskew,Sim_rank,weight,relat_weight,cumsum,lcv_x_weight,lskew_x_weight
0,23001,64,0.161115,0.147465,1.000000,64.000000,0.546684,64,10.311388,9.437747
1,23003,60,0.178319,0.207121,0.653179,39.190751,0.334765,124,6.988471,8.117237
2,25009,49,0.178077,-0.034361,0.283237,13.878613,0.118550,173,2.471464,-0.476881


## 6. Calculate parameters using LC-V and L-SKEW weighted 

In [ ]:
lcv_pool = pool_final['lcv_x_weight'].sum()/pool_final['weight'].sum()
lskew_pool = pool_final['lskew_x_weight'].sum()/pool_final['weight'].sum()


param_pool = distr.gev.lmom_fit(lmom_ratios=[1, lcv_pool, lskew_pool])
param_pool


OrderedDict([('c', 0.03777077631628866),
             ('loc', 0.8636400853701754),
             ('scale', 0.2520041249684976)])

## 7. Flood estimation


In [ ]:
# add the missing part of the API url
station_API_url_amx = f'data-type=amax-flow&station={set_pool.stn_id}'

# fetch data and create dataframes
pool_main_stn_amax = fetch_station_data(station_API_url_amx)

# Calculate QMED
qmed_pool_single_site = qmed_single_site_amax(pool_main_stn_amax)

# Taken the parameters of the fit distribution
est_flow_pool = genextreme.ppf(1 - np.array(0.01), **param_pool)
# Multiply the est_flow by QMED to obtained the final flood estimation
final_flows_pool = est_flow_pool * qmed_pool_single_site
print('-------------------------------------')
print(f'The Flood Estimation for 100 years for Station: {set_pool.stn_id} is {final_flows_pool}')
print('-------------------------------------')

-------------------------------------
The Flood Estimation for 100 years for Station: 23001 is 1622.8210481599572
-------------------------------------


## 8. Graphical Representation

In [ ]:
graph_pool(pool_main_stn_amax, 'gev', [param, param_pool])

# END OF NOTEBOOK
---
---